In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd #Basic library for all of our dataset operations
import numpy as np
import requests
import io
import statsmodels.tsa.api as smt
import statsmodels as sm
import tensorflow as tf
import pmdarima as pm
import warnings
import xgboost as xgb

import lightgbm as lgb
import gluonts
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
# # import shap
# # warnings.filterwarnings("ignore") #We will use deprecated models of statmodels which throw a lot of warnings to use more modern ones

from utils.metrics import evaluate
from utils.plots import bar_metrics

from statsmodels.tsa.ar_model import AR
from random import random
from datetime import datetime
from fbprophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn import linear_model, svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV
from math import sqrt
from xgboost import plot_importance, plot_tree
from statsmodels.tsa.arima_model import ARIMA
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from itertools import islice
from pylab import rcParams
# progress bar
from tqdm import tqdm, tqdm_notebook
# from bayes_opt import BayesianOptimization


#Extra settings
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
plt.style.use('bmh')
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['text.color'] = 'k'
print(tf.__version__)

Importing plotly failed. Interactive plots will not work.


2.4.0


In [114]:
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator

In [116]:
from gluonts.model.deepvar import DeepVAREstimator

In [58]:
import mxnet as mx

In [2]:
#Third-party imports
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from gluonts.model import deepar
from gluonts.dataset import common
from gluonts.dataset.util import to_pandas
from gluonts.model.predictor import Predictor

In [3]:
raw_col = ['date', 'highest', 'high', 'mid', 'low', 'ave', 'ave_dif', 'volume', 'vol_dif', 'resid']
raw = pd.read_csv('FS443.csv',names=raw_col).drop(['ave_dif', 'vol_dif', 'resid'], axis=1)
for i in range(raw.shape[0]):
    raw.volume[i] = int(raw.volume[i].split(' ')[0].replace(',', ''))
raw.volume = raw.volume.values.astype('int')

ts_date = raw.copy().date
for itr in range(raw.shape[0]):
    date = raw.date[itr].split('/')
    date[0] = str(int(date[0]) + 1911)
    result = '/'.join(date)
    ts_date[itr] =  np.datetime64(datetime.strptime(result, "%Y/%m/%d").date())
    


ts = raw.copy()
ts.date = ts_date
ts['date'] = pd.to_datetime(ts['date'], errors='coerce')
ts = ts.set_index('date')
ts.index = pd.to_datetime(ts.index)
ts

,highest,high,mid,low,ave,volume
date,,,,,,
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-06,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2020-11-26,201,201,137,67,134.9,97
2020-11-27,170,170,112,52,111.8,176
2020-11-28,171,171,170,152,166.7,81


In [4]:
datelist = pd.date_range(ts.index[0], periods=ts.shape[0]).tolist()
ts.index = datelist

In [5]:
train = ts[:-14]
test = ts[-14:]

In [5]:
train

,highest,high,mid,low,ave,volume
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-05,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2017-01-20,161,159,111,57,109.7,299
2017-01-21,142,142,54,54,71.6,75
2017-01-22,155,145,85,52,90.2,160
2017-01-23,162,151,95,61,99.3,95


In [6]:
train_data = common.ListDataset([{
    "start": train.index[0],
   "target": train.ave.values, 
    "feat_dynamic_real": [train.highest.values, train.high.values, train.mid.values, train.low.values, train.volume.values]}], 
    freq="D")

In [7]:
test_data = common.ListDataset([{
    "start": ts.index[0],
   "target": ts.ave.values, 
    "feat_dynamic_real": [ts.highest.values, ts.high.values, ts.mid.values, ts.low.values, ts.volume.values]}], 
    freq="D")

### Uni DeepAR

In [51]:
train_data = common.ListDataset([{
    "start": train.index[0],
   "target": train.ave.values, 
}], 
    freq="D")

In [52]:
estimator = deepar.DeepAREstimator(freq="D", prediction_length=14)
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00, 10.29it/s, epoch=100/100, avg_epoch_loss=4.35]


In [56]:
mse = ((test.ave.values - prediction.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 55.23


### Multi DeepAR

In [62]:
estimator = deepar.DeepAREstimator(freq="D", prediction_length=14,use_feat_dynamic_real=True,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████████████████████████████████████████| 50/50 [00:07<00:00,  7.00it/s, epoch=136/200, avg_epoch_loss=2.93]


100%|██████████████████████████████████████████████| 50/50 [00:07<00:00,  7.06it/s, epoch=200/200, avg_epoch_loss=2.86]


In [101]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

In [103]:
ts_entry = tss[0]

In [104]:
forecast_entry = forecasts[0]

In [112]:
test.ave.values

array([ 80.7, 116.6, 103.3,  93.6, 100.5, 113.7, 137.7, 140.7, 169.4,
       134.9, 111.8, 166.7, 122.4, 200.5])

In [111]:
forecast_entry.mean

array([ 96.171974, 100.859436,  69.44821 ,  93.74693 ,  95.328606,
       139.58736 , 138.75697 , 114.09435 , 127.28977 , 108.67466 ,
        71.44339 ,  93.56921 ,  99.03169 ,  94.16289 ], dtype=float32)

In [113]:
mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 41.69


### Simple feedfoward

In [115]:
estimator = SimpleFeedForwardEstimator(freq="D", prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|███████████████████████████████████████████████| 50/50 [00:01<00:00, 33.22it/s, epoch=86/200, avg_epoch_loss=5.14]


In [117]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 43.59


In [31]:
from gluonts.model.transformer import TransformerEstimator

In [32]:
estimator = TransformerEstimator(freq="D", prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00, 11.38it/s, epoch=136/200, avg_epoch_loss=4.48]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00, 11.87it/s, epoch=149/200, avg_epoch_loss=4.46]


In [125]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 30.43


In [126]:
from gluonts.model.canonical import CanonicalRNNEstimator

In [128]:
estimator = CanonicalRNNEstimator(freq="D",context_length =14, prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|█████████████████████████████████████████████| 50/50 [00:01<00:00, 25.36it/s, epoch=136/200, avg_epoch_loss=0.526]


100%|█████████████████████████████████████████████| 50/50 [00:01<00:00, 26.78it/s, epoch=200/200, avg_epoch_loss=0.285]


In [132]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 43.9


In [133]:
from gluonts.model.deep_factor import DeepFactorEstimator

In [134]:
estimator = DeepFactorEstimator(freq="D", prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|█████████████████████████████████████████████| 50/50 [00:01<00:00, 25.48it/s, epoch=136/200, avg_epoch_loss=0.706]


100%|█████████████████████████████████████████████| 50/50 [00:01<00:00, 25.68it/s, epoch=200/200, avg_epoch_loss=0.659]


In [135]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 58.04


In [15]:
from gluonts.model.deepstate import DeepStateEstimator

In [ ]:
estimator = DeepStateEstimator(freq="D",cardinality=[1],  use_feat_static_cat=False,prediction_length=14,use_feat_dynamic_real=True,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


 52%|███████████████████████▉                      | 26/50 [00:20<00:18,  1.28it/s, epoch=1/200, avg_epoch_loss=-.0847]

In [ ]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

In [143]:
estimator = DeepStateEstimator(freq="D",cardinality=[0],  use_feat_static_cat=False,prediction_length=14,use_feat_dynamic_real=True,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|█████████████████████████████████████████████| 50/50 [00:26<00:00,  1.86it/s, epoch=136/200, avg_epoch_loss=-3.71]


100%|█████████████████████████████████████████████| 50/50 [00:26<00:00,  1.91it/s, epoch=200/200, avg_epoch_loss=-3.85]


In [144]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 41.8


In [12]:
from gluonts.model.gp_forecaster import GaussianProcessEstimator

In [13]:
estimator = GaussianProcessEstimator(freq="D",cardinality=1,  prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

Multiprocessing is not supported on Windows, num_workers will be set to None.
  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████████████████████████████████████████| 50/50 [00:01<00:00, 31.31it/s, epoch=136/200, avg_epoch_loss=65.1]


100%|██████████████████████████████████████████████| 50/50 [00:01<00:00, 31.02it/s, epoch=200/200, avg_epoch_loss=64.8]


In [14]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 44.36


In [16]:
from gluonts.model.npts import NPTSEstimator 

In [17]:
estimator = NPTSEstimator (freq="D", prediction_length=14)
predictor = estimator.train(training_data=train_data)

In [18]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 40.45


In [19]:
from gluonts.model.seq2seq import MQCNNEstimator

In [20]:
estimator = MQCNNEstimator(freq="D",  prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

Multiprocessing is not supported on Windows, num_workers will be set to None.
  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|████████████████████████████████████████████| 50/50 [00:11<00:00,  4.38it/s, epoch=136/200, avg_epoch_loss=0.0652]


100%|████████████████████████████████████████████| 50/50 [00:11<00:00,  4.49it/s, epoch=200/200, avg_epoch_loss=0.0555]


In [21]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 42.24


In [22]:
from gluonts.model.seq2seq import MQRNNEstimator

In [23]:
estimator = MQRNNEstimator(freq="D",  prediction_length=14,trainer=Trainer(ctx='cpu', epochs=200, learning_rate=1e-3))
predictor = estimator.train(training_data=train_data)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|████████████████████████████████████████████| 50/50 [00:06<00:00,  7.16it/s, epoch=136/200, avg_epoch_loss=0.0552]


100%|████████████████████████████████████████████| 50/50 [00:06<00:00,  7.30it/s, epoch=200/200, avg_epoch_loss=0.0245]


In [24]:


forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

tss = list(ts_it)
forecasts = list(forecast_it)

ts_entry = tss[0]
forecast_entry = forecasts[0]

mse = ((test.ave.values - forecast_entry.mean) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

The Root Mean Squared Error of our forecasts is 42.09


In [ ]:
from gluonts.model.seq2seq import Seq2SeqEstimator